# k-Nearest Neighbor (kNN) on CIFAR-10

이 노트북은 과제 요구사항에 맞게 `KNearestNeighbor` 클래스를 사용하여 CIFAR-10 데이터셋 분류를 수행합니다.

## 체크리스트
- [x] Conda 환경: `ssu_knn` (`python=3.10`)
- [x] 라이브러리: `numpy`, `opencv-python`, `Pillow`, `matplotlib`
- [x] `k_nn.py` 구현 (두 루프 / 한 루프 / 무루프 거리 계산)
- [x] k 값 스윕, 검증셋으로 최적 k 선택
- [x] 테스트셋 정확도 측정
- [x] 거리계산 방식별 속도 비교


In [ ]:
# 환경 점검
import sys, os, time, pickle
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

from k_nn import KNearestNeighbor, accuracy, train_val_split

print(sys.version)
print('numpy', np.__version__)

## CIFAR-10 로드
아래 코드는 **CIFAR-10 python 버전** 디렉토리(`cifar-10-batches-py/`)가 로컬에 있는 경우를 가정합니다. 
과제 저장소에서 제공하는 유틸이 있다면 그 함수를 사용해도 됩니다.

- 학습: `data_batch_1`~`data_batch_5`
- 테스트: `test_batch`


In [ ]:
def load_cifar10(root: str):
    root = Path(root)
    def _load_batch(p):
        with open(p, 'rb') as f:
            d = pickle.load(f, encoding='latin1')
        data = d['data']  # (N, 3072) uint8
        labels = d['labels']  # list of ints
        return np.array(data, dtype=np.uint8), np.array(labels, dtype=np.int64)

    X_list, y_list = [], []
    for i in range(1, 6):
        X_i, y_i = _load_batch(root / f'data_batch_{i}')
        X_list.append(X_i); y_list.append(y_i)
    X_train = np.concatenate(X_list, axis=0)
    y_train = np.concatenate(y_list, axis=0)
    X_test, y_test = _load_batch(root / 'test_batch')
    return X_train, y_train, X_test, y_test

DATA_DIR = 'cifar-10-batches-py'  # 필요 시 경로 수정
if Path(DATA_DIR).exists():
    Xtr_u8, ytr, Xte_u8, yte = load_cifar10(DATA_DIR)
else:
    # 경로가 없다면 데모용 난수 데이터 사용 (코드 확인용)
    print('[WARN] CIFAR-10 경로를 찾을 수 없습니다. 데모 데이터를 사용합니다.')
    Ntr, Nte, D = 2000, 400, 3072
    rng = np.random.default_rng(0)
    Xtr_u8 = rng.integers(0, 256, size=(Ntr, D), dtype=np.uint8)
    ytr = rng.integers(0, 10, size=(Ntr,), dtype=np.int64)
    Xte_u8 = rng.integers(0, 256, size=(Nte, D), dtype=np.uint8)
    yte = rng.integers(0, 10, size=(Nte,), dtype=np.int64)

print('Train:', Xtr_u8.shape, ytr.shape, 'Test:', Xte_u8.shape, yte.shape)

## 전처리 & 분할
- 벡터화: (N, 3072) 그대로 사용
- 정규화(선택): 평균/표준편차 스케일링은 보통 kNN에 이득


In [ ]:
Xtr = Xtr_u8.astype(np.float32)
Xte = Xte_u8.astype(np.float32)
mean = np.mean(Xtr, axis=0, keepdims=True)
std = np.std(Xtr, axis=0, keepdims=True) + 1e-7
Xtr = (Xtr - mean) / std
Xte = (Xte - mean) / std

X_train, y_train, X_val, y_val = train_val_split(Xtr, ytr, val_ratio=0.2, seed=42)
print('Train:', X_train.shape, y_train.shape)
print('Val   :', X_val.shape, y_val.shape)

## kNN 학습 (메모라이즈)


In [ ]:
knn = KNearestNeighbor()
knn.train(X_train, y_train)

## 거리 계산 방식 속도 비교
⚠️ 전체 CIFAR-10(50k x 10k)은 오래 걸릴 수 있으니, 검증용으로 작은 서브샘플을 사용하세요.


In [ ]:
def benchmark(num_loops, k=5, n_te=200):
    X_sub = X_val[:n_te]
    y_sub = y_val[:n_te]
    t0 = time.time()
    y_pred = knn.predict(X_sub, k=k, num_loops=num_loops)
    dt = time.time() - t0
    acc = accuracy(y_sub, y_pred)
    return dt, acc

results = {}
for loops in [2, 1, 0]:
    dt, acc = benchmark(loops, k=5)
    results[loops] = (dt, acc)
    print(f"num_loops={loops} | time={dt:.3f}s | acc={acc:.4f}")

## k 값 스윕 (검증셋)


In [ ]:
k_candidates = [1, 3, 5, 7, 9, 11, 13, 15]
val_accs = []
for k in k_candidates:
    y_pred = knn.predict(X_val, k=k, num_loops=0)
    acc = accuracy(y_val, y_pred)
    val_accs.append(acc)
    print(f"k={k:2d} | val acc={acc:.4f}")

best_k = k_candidates[int(np.argmax(val_accs))]
print('Best k:', best_k)

### 시각화


In [ ]:
plt.figure()
plt.plot(k_candidates, val_accs, marker='o')
plt.title('Validation Accuracy vs. k')
plt.xlabel('k')
plt.ylabel('Accuracy')
plt.grid(True)
plt.show()

## 테스트셋 성능


In [ ]:
knn_full = KNearestNeighbor()
knn_full.train(Xtr, ytr)
yte_pred = knn_full.predict(Xte, k=best_k, num_loops=0)
test_acc = accuracy(yte, yte_pred)
print('Test accuracy:', test_acc)

os.makedirs('output', exist_ok=True)
import json
with open('output/val_results.json', 'w') as f:
    json.dump({'k_candidates': k_candidates,
               'val_accs': [float(a) for a in val_accs],
               'best_k': int(best_k),
               'test_acc': float(test_acc)}, f, indent=2)
print('Saved: output/val_results.json')

## 노트
- 거리 계산 방식은 **정확도가 동일**해야 합니다 (동일한 거리 정의/전처리인 경우).
- 속도는 보통 `no_loops` > `one_loop` > `two_loops` 순입니다.
- 메모리 한계에 부딪히면, 검증/테스트 시 **배치 단위**로 예측하는 함수를 추가해도 됩니다.
